# ONS Charts

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/covid-stats/

## Imports

Standard python libraries plus determination of projdir, basic printable class, etc

In [1]:
import os

import unittest

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as tck

import common_core
import ons_core

## NumPy Helper Functions

Useful functionality such as moving average or rolling sum

In [2]:
def shiftRegistrations(data):
    """Shift registration data left by half a period"""
    
    # Final value is invalid (so not included in the convolution result) and needs to be zero
    result = np.append(np.convolve(data, np.array([0.5, 0.5]), mode="valid"), 0)
    
    return result

In [3]:
class TestShiftRegistrations(unittest.TestCase):
    '''Class to test rollingSum function'''   

    def testShift(self):
        '''Test processing of a list shorter than the window size'''

        actual = shiftRegistrations(np.arange(6))
        expected = np.array([0.5, 1.5, 2.5, 3.5, 4.5, 0])

        self.assertEqual((actual == expected).all(), True)
        
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.042s

OK


## Plot Data

Simple plots of ONS data

In [4]:
def prunePoints(x_points, y_points):
    '''Remove leading and trailing zeros'''

    start = np.where(y_points > 0)[0][0]
    end = np.where(y_points > 0)[0][-1]
    
    return x_points[start:end + 1], y_points[start:end + 1]
    

def plotRegion(cache, areaName, fig, ax, startDate=None, stopDate=None):
    '''Plot data for visual inspection'''
    
    areaData = cache[areaName]

    ax.set_title(f"Weekly Deaths in {areaName}")
    ax.set_ylabel('Number of deaths')

    # Determine which rows need to be plotted
    startIdx = np.where(areaData[ons_core.WEEK_ENDED] >= startDate)[0][0]
    stopIdx = np.where(areaData[ons_core.WEEK_ENDED] < stopDate)[0][-1]

    xMin = yMin = stopIdx
    xMax = yMax = 0
    
    y_points = areaData[ons_core.TOTAL_REGISTRATIONS][startIdx:stopIdx+1]
    x_points = np.arange(len(y_points))   
    x_points, y_points = prunePoints(x_points, y_points)
    ax.plot(x_points, y_points, label = "Total Registrations", color='navy', linestyle="dotted")
    xMax = max(xMax, max(x_points))
    yMax = max(yMax, max(y_points))
    
    y_points = areaData[ons_core.TOTAL_OCCURRENCES][startIdx:stopIdx+1]
    x_points = np.arange(len(y_points))       
    x_points, y_points = prunePoints(x_points, y_points)
    ax.plot(x_points, y_points, label = "Total Occurrences", color='navy')
    xMin = min(xMin, min(x_points))
    yMin = min(yMin, min(y_points))
    xMax = max(xMax, max(x_points))
    yMax = max(yMax, max(y_points))

    y_points = areaData[ons_core.COVID_REGISTRATIONS][startIdx:stopIdx+1]
    x_points = np.arange(len(y_points))       
    x_points, y_points = prunePoints(x_points, y_points)
    ax.plot(x_points, y_points, label = "COVID-19 Registrations", color='red', linestyle="dotted")
    xMin = min(xMin, min(x_points))
    yMin = min(yMin, min(y_points))
    xMax = max(xMax, max(x_points))
    yMax = max(yMax, max(y_points))

    y_points = areaData[ons_core.COVID_OCCURRENCES][startIdx:stopIdx+1]
    x_points = np.arange(len(y_points))       
    x_points, y_points = prunePoints(x_points, y_points)
    ax.plot(x_points, y_points, label = "COVID-19 Occurrences", color='red')
    xMin = min(xMin, min(x_points))
    yMin = min(yMin, min(y_points))
    xMax = max(xMax, max(x_points))
    yMax = max(yMax, max(y_points))

    # Determine y-axis labelling
    yMax *= 1.1
    if yMax > 10000:
        interval = yMax // 10 // 1000 * 1000
    elif yMax > 1000:
        interval = yMax // 10 // 100 * 100
    else:
        interval = yMax // 10

    ax.set_ylim(ymin=0, ymax=yMax)
    ax.set_yticks(np.arange(0, yMax, interval))
    ax.get_yaxis().set_major_formatter(tck.FuncFormatter(lambda x, p: format(int(x), ',')))

    # Determine x-axis labelling
    xticks = np.array(areaData[ons_core.WEEK_ENDED])[startIdx:stopIdx+1]
    tickInterval = (stopIdx - startIdx) // 52

    ax.set_xticks(np.arange(0, len(xticks), tickInterval))
    ax.set_xticklabels(xticks[::tickInterval], rotation=90)

    ax.legend()


def plotRegions(cache, areaNames, startDate=None, stopDate=None):
    '''Plot data for visual inspection'''
    
    # week_ended, week_number, total_registrations, covid_registrations, covid_occurrences
    # week_ended, week_number, total_deaths

    fig, axs = plt.subplots(len(areaNames), figsize=(16, 6 * len(areaNames)))
    
    for i in range(len(areaNames)):
        areaName = areaNames[i]
        areaData = cache[areaName]

        if len(areaNames) == 1:
            plotRegion(cache, areaName, fig, axs, startDate=startDate, stopDate=stopDate)
        else:
            plotRegion(cache, areaName, fig, axs[i], startDate=startDate, stopDate=stopDate)

    plt.subplots_adjust(hspace=0.5)
    plt.show(fig)

## Interactive Testing

In [5]:
if __name__ == '__main__':
    
    verbose = False
    
    cache = ons_core.loadCsvFiles(ons_core.ONS_DEATHS, "weekly", verbose = verbose)

    areaNames = ["England", "London"] #, "South East", "East of England"]
    #areaNames = ["England", "Wales"]

    startDate = '2000-01-01'
    stopDate = '2020-12-31'

    #plotRegions(cache, areaNames, startDate=startDate, stopDate=stopDate)